In [2]:
import pandas as pd
from scipy.stats import pearsonr
import math


# Lee los primeros 100 resultados de conservación predichos
#file_name = "output_results_curated_8M.csv"
file_name = "output_results_curated_8M.csv"
predicted_df = pd.read_csv(file_name)
print("predicted_df", predicted_df)




predicted_df                                                  header  \
0      A0A1Y2ANA8.1/421-668 Pfam=PF00003.25 type=domain   
1           V3ZBP4.1/12-278 Pfam=PF00003.25 type=domain   
2      A0A226EIY0.1/621-830 Pfam=PF00003.25 type=domain   
3      A0A6J1SJU2.1/386-517 Pfam=PF00004.32 type=domain   
4          Q54ST1.1/607-740 Pfam=PF00004.32 type=domain   
...                                                 ...   
35866      A0A5B9Q753.1/5-47 Pfam=PF20607.1 type=family   
35867      A0A3M1R399.1/5-47 Pfam=PF20607.1 type=family   
35868     A0A7K3NCN5.1/12-97 Pfam=PF20619.1 type=family   
35869      A0A1M6TTR7.1/1-91 Pfam=PF20619.1 type=family   
35870     A0A5J5ICB5.1/11-99 Pfam=PF20619.1 type=family   

                                                sequence  \
0      KWTDIPAIIIGGISVVGIIITLVVFFITVKNRNALVIRRSSPLFLY...   
1      AVMSWIIGCVIALVSLVYLVLNIKLRNTRLIKMSSPNLNCLVASGG...   
2      YLNPLAMAAMIFSGFGILLALFVLKVFWRYNDTPIIKAAGRELSYV...   
3      VLLFGPSGTGKTMLAHALAQDSAASVHTLIG

In [3]:
true_df = pd.read_csv("curated_dataset/conservation_scores.csv")
#true_df = true_df.groupby(true_df.columns[0]).head(1).head(100).reset_index(drop=True)

df = pd.DataFrame(true_df)

# Convertir 'conservation score' a cadena antes de unir
df['conservation score'] = df['conservation score'].astype(str)

# Agrupar y concatenar los valores de 'conservation score'
true_df = df.groupby('sequence id')['conservation score'].agg(' '.join).reset_index()

print(true_df)

                sequence id                                 conservation score
0       A0A009J727.1/56-242  0.500488 0.436523 0.448975 0.209595 0.193726 0...
1      A0A009KMN5.1/230-429  0.329834 0.481934 0.342285 0.564941 0.238525 0...
2      A0A010NKV5.1/319-434  0.318115 0.44165 0.150513 0.206177 0.512695 0....
3        A0A010QH20.1/5-204  0.571289 0.745605 0.554688 0.400146 0.46582 0....
4      A0A010YGC5.1/692-820  0.056244 0.05899 0.143555 0.117065 0.127563 0....
...                     ...                                                ...
35866         Z9JGP6.1/5-79  0.178955 0.203369 0.152954 0.325195 0.210815 0...
35867        Z9JGW4.1/23-78  0.232544 0.265381 0.25415 0.260742 0.244019 0....
35868        Z9JK55.1/3-273  0.025726 0.046112 0.07605 0.068909 0.122986 0....
35869       Z9JRB9.1/29-175  0.128906 0.202637 0.273438 0.494873 0.277588 0...
35870         Z9JX40.1/5-99  0.375 0.76416 0.414551 0.384277 0.436768 0.251...

[35871 rows x 2 columns]


In [4]:
# Extrae la parte común de las secuencias (puedes ajustar esto según tus necesidades)
predicted_df['sequence_id'] = predicted_df['header'].str.extract(r'([A-Za-z0-9]+\.[0-9]+/[0-9]+-[0-9]+)')
true_df['sequence_id'] = true_df['sequence id'].str.extract(r'([A-Za-z0-9]+\.[0-9]+/[0-9]+-[0-9]+)')


print(predicted_df)
print(true_df)

# Fusionamos los DataFrames por la columna 'common_sequence'
combined_df = pd.merge(predicted_df, true_df, how='inner',
                       left_on='sequence_id', right_on='sequence_id')




                                                 header  \
0      A0A1Y2ANA8.1/421-668 Pfam=PF00003.25 type=domain   
1           V3ZBP4.1/12-278 Pfam=PF00003.25 type=domain   
2      A0A226EIY0.1/621-830 Pfam=PF00003.25 type=domain   
3      A0A6J1SJU2.1/386-517 Pfam=PF00004.32 type=domain   
4          Q54ST1.1/607-740 Pfam=PF00004.32 type=domain   
...                                                 ...   
35866      A0A5B9Q753.1/5-47 Pfam=PF20607.1 type=family   
35867      A0A3M1R399.1/5-47 Pfam=PF20607.1 type=family   
35868     A0A7K3NCN5.1/12-97 Pfam=PF20619.1 type=family   
35869      A0A1M6TTR7.1/1-91 Pfam=PF20619.1 type=family   
35870     A0A5J5ICB5.1/11-99 Pfam=PF20619.1 type=family   

                                                sequence  \
0      KWTDIPAIIIGGISVVGIIITLVVFFITVKNRNALVIRRSSPLFLY...   
1      AVMSWIIGCVIALVSLVYLVLNIKLRNTRLIKMSSPNLNCLVASGG...   
2      YLNPLAMAAMIFSGFGILLALFVLKVFWRYNDTPIIKAAGRELSYV...   
3      VLLFGPSGTGKTMLAHALAQDSAASVHTLIGSELFSKVLGETDA

In [5]:
# Convertimos los valores de 'conservation' y 'conservation score' a listas de números
combined_df['conservation'] = combined_df['conservation'].apply(lambda x: [float(val) for val in x.split()])
combined_df['conservation score'] = combined_df['conservation score'].apply(lambda x: [float(val) for val in x.split()])

# Eliminamos filas que contienen infinitos o NaNs
combined_df = combined_df[~combined_df['conservation'].apply(lambda x: any(math.isinf(val) or math.isnan(val) for val in x))]
combined_df = combined_df[~combined_df['conservation score'].apply(lambda x: any(math.isinf(val) or math.isnan(val) for val in x))]

# Calculamos el coeficiente de Pearson por residuo
pearson_coefficient, _ = pearsonr(
    combined_df['conservation'].explode(), combined_df['conservation score'].explode())
print(f"Coeficiente de Pearson por residuo: {pearson_coefficient:.4f}")

Coeficiente de Pearson por residuo: 0.3829
